<a href="https://colab.research.google.com/github/qa21336/HousingStatisticsAnalysis-/blob/main/populationDataWrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ADS Data Wrangling for Population Dataset

### Explore the dataset

In [ ]:
import pandas as pd

popDf = pd.read_csv("populationDatasets/MYEB1_detailed_population_estimates_series_UK_(2020_geog21).csv")

# We dont need Local Area name as well as code
popDf = popDf.drop("laname21", axis=1)
popDf.head()

,ladcode21,country,sex,age,population_2001,population_2002,population_2003,population_2004,population_2005,population_2006,...,population_2011,population_2012,population_2013,population_2014,population_2015,population_2016,population_2017,population_2018,population_2019,population_2020
0,E06000001,E,1,0,525,502,516,553,531,583,...,601,595,576,551,497,576,507,535,515,517
1,E06000001,E,1,1,544,526,498,526,550,539,...,573,615,601,567,553,512,570,515,532,519
2,E06000001,E,1,2,548,548,551,507,521,539,...,580,586,621,600,576,562,534,576,515,528
3,E06000001,E,1,3,556,546,548,553,508,514,...,600,578,584,625,598,579,565,530,583,520
4,E06000001,E,1,4,606,549,553,540,554,512,...,600,616,593,568,620,604,591,576,533,580


In [ ]:
import numpy as np

# Find age range for restructured data
print("youngest age is: "+str(np.min(popDf.loc[:, "age"].to_numpy())))
print("oldest age is: "+str(np.max(popDf.loc[:, "age"].to_numpy())))

youngest age is: 0
oldest age is: 90


### Extract important information from columns and create place for restructured data

In [ ]:
# Get countries
countries = np.unique(popDf.loc[:, "country"].to_numpy())

# Get population columns and years
population_cols = list(filter(lambda s: s[0] == "p", popDf.columns))
years = list(map(lambda col: col.split("_")[1], population_cols))

# Create place for restructured data
newPopDict = {}
newPopDict["country"] = []
newPopDict["year"] = []
newPopDict["population"] = []

for i in range(0, 100, 10):
    colName = "age_"+str(i)+"-"+str(i+10)+"_proportion"
    newPopDict[colName] = []

print(newPopDict)

### Fill in Population Sums

In [ ]:
# Extract sum of populations in each year
for country in countries:
    rowsForCountry = popDf.loc[popDf["country"] == country]

    # Get sum for each year
    for year, popCol in zip(years, population_cols):
        sum = np.sum(rowsForCountry.loc[:, popCol].to_numpy())

        # Update newPopDf
        newPopDict["country"].append(country)
        newPopDict["year"].append(year)
        newPopDict["population"].append(sum)

### Fill in Age Ratios

In [ ]:
# Extract proportion of population in each decade of their lives
for country in newPopDict["country"]:
    rowsForCountry = popDf.loc[popDf["country"] == country]

    # Get proportion for each year
    for year, popCol in zip(years, population_cols):
        cellsForCountryYear = rowsForCountry.loc[:, ["age", popCol]].to_numpy()

        # Get Ages
        for age in range(0, 100, 10):
            agePopsInRange = np.array(list(filter(lambda agePop: age <= agePop[0] and agePop[0] < age + 10,
                                                  cellsForCountryYear)))
            print(np.sum(agePopsInRange, axis=0)[1])
            break
        break
    break


6044627


In [ ]:
newPopDf = pd.DataFrame(data=newPopDict)
newPopDf